# Filtering your geocoding results
In this guide we will learn how to narrow down and filter your geocoding results using the following techniques:
 - [Specify the output spatial reference](#specify-the-output-spatial-reference)
 - [Specify the maximum number of candidates](#specify-the-maximum-number-of-candidates)
 - [SEarch within an extent](#search-within-an-extent)
 - [Proximity searches](#proximity-searches)
 - [category filtering](#category-filtering)

<a id="specify-the-output-spatial-reference"></a>
## Specify the output spatial reference

By default the World Geocoding Service returns candidate geometry in WGS84 coordinates (decimal degrees). You can specify a different spatial reference for output coordinates by using the outSR parameter. This is necessary if you have a mapping application in which you display geocoding candidates and the map spatial reference is not WGS84. For example, the ArcGIS.com basemaps use a Web Mercator spatial reference, with coordinates in meters. In order to display geocoding candidates correctly in such a map you would need to set outSR=102100, which is the well-known ID (WKID) of the Web Mercator spatial reference.

For a list of valid WKID values, see <a href="http://resources.arcgis.com/EN/HELP/REST/APIREF/PCS.HTML">Projected Coordinate Systems</a> and <a href="http://resources.arcgis.com/EN/HELP/REST/APIREF/GCS.HTML">Geographic Coordinate Systems</a>.

### Example: Specify output coordinates in Web Mercator spatial reference (380 new york st redlands ca)

In [49]:
esrihq = geocode("380 New York St, Redlands, CA")[0]

In [50]:
esrihq['location']

{'x': -117.19566636800778, 'y': 34.05649035814457}

The location above is in the default spatial reference of the geocoder (4326). We can set the output spatial reference to 102100 using the outSR parameter to obtain the location in the Web Mercator spatial reference(102100):

In [51]:
esrihq2 = geocode("380 New York St, Redlands, CA", out_sr=102100)[0]

In [52]:
esrihq2['location']

{'x': -13046161.903265003, 'y': 4036389.8256044337}

<a id="specify-the-maximum-number-of-candidates"></a>
## Specify the maximum number of candidates

The maxLocations parameter allows you to specify the maximum number of candidates to be returned by a search, up to the maximum number of candidates allowed by the World Geocoding Service. By default the service allows up to 20 candidates to be returned for address searches, and 50 for POI searches. As an example, if you set max_locations=10, then geocode() will return the top 10 candidates for the search, regardless of whether the search matches to an address or a POI. If no value is specified for max_locations, then geocode() returns upto a maximum default of 20 matches.

### Example: Specify the maximum number of candidates for a POI search (Starbucks in Paris)

In [53]:
matches = geocode("coffee shops austin", out_fields="PlaceName,City,Country", max_locations=3)

We get back 3 matches as we've specified max_locations=3:

In [54]:
len(matches)

3

In [55]:
matches

[{'address': 'Starbucks',
  'attributes': {'City': 'Austin', 'Country': 'USA', 'PlaceName': 'Starbucks'},
  'extent': {'xmax': -97.963419,
   'xmin': -97.973419,
   'ymax': 30.346169,
   'ymin': 30.336169},
  'location': {'x': -97.96841805299965, 'y': 30.34116889100045},
  'score': 100},
 {'address': 'Starbucks',
  'attributes': {'City': 'Austin', 'Country': 'USA', 'PlaceName': 'Starbucks'},
  'extent': {'xmax': -97.928381,
   'xmin': -97.938381,
   'ymax': 30.30853,
   'ymin': 30.29853},
  'location': {'x': -97.93337966199965, 'y': 30.30352973100048},
  'score': 100},
 {'address': 'Starbucks',
  'attributes': {'City': 'Austin', 'Country': 'USA', 'PlaceName': 'Starbucks'},
  'extent': {'xmax': -97.873198,
   'xmin': -97.883198,
   'ymax': 30.207829,
   'ymin': 30.197829},
  'location': {'x': -97.87819655899966, 'y': 30.20282866700046},
  'score': 100}]

<a id="search-within-an-extent"></a>
## Search within an extent

The geocode method allows spatial filtering of search results by using the searchExtent parameter. If you want to confine a search to a localized area, something that is especially useful in a mobile application, you can define a bounding rectangle to search within. No candidates outside of the rectangle are returned. Bounding rectangle coordinates can be entered as a simple comma-separated string in the format &lt;lower left corner&gt;,&lt;upper right corner&gt;. If the simple format is used, the coordinates must be in the default spatial reference of the geocoder, which is WGS84. The searchExtent parameter can be used with all supported search types (street address, POI, admin place, postal code).

The example URL below illustrates how to find McDonald's in downtown San Diego using the simple searchExtent format.

### Example: Find POIs using searchExtent with default spatial reference (McDonald's)
The example below finds and plots upto 100 Mc Donalds' locations in San Diego, CA. The extent parameter is obtained from the geocoder's geocoding result for San Diego (sandiego['extent']) and passed into the next geocode() request for 100 Mc Donalds' locations:

In [56]:
sandiego = geocode("San Diego, CA")[0]
map = gis.map("San Diego, CA")
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_11.png)

In [57]:
# find and plot upto 100 Mc Donalds' locations in San Diego, CA

mcds = geocode("McDonalds", sandiego['extent'],  max_locations=100)
for mcd in mcds:
    map.draw(mcd['location'])

### Example: Find POIs using searchExtent and zone (Starbucks garland)

Requests that include searchExtent can also include zone information (that is, city, state, and country). If the extent defined for searchExtent is large enough to encompass multiple cities, it may be necessary to include the city name in the search to achieve optimal results. For example, if the searchExtent covers the entire state of Texas, and you search for Starbucks, there could be matches returned in any city in Texas. If you specifically want to find Starbucks in Garland for example, then this needs to be specified in the search.

In [58]:
texas = geocode("TX")[0]
map = gis.map("Dallas Fort Worth", 10)
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/geocoding-57.png)

In [59]:
starbucks = geocode("Starbucks Garland", texas['extent'], max_locations=20)
for starbuck in starbucks:
    map.draw(starbuck['location'])

### Example: Find a street address using search_extent (380 New York St)

You can also search for street addresses within an extent. When the search_extent parameter is defined for an address search, city and postal code can be omitted from the search and valid matches can still be found. However, if the search_extent is large, it is possible for a street address to occur multiple times within the extent, and it may be necessary to refine the search by including city, state, postal code, or other distinguishing information. Additionally, if the search includes a city or postal code that is outside the search_extent, no matches will be returned. See the following example which illustrates finding a street address using search_extent.

In [60]:
map = gis.map("Redlands, CA")
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/geocoding-3.PNG)

In [61]:
redlands = geocode("Redlands, CA")[0]

# search for "380 New Yort St" within the extent of Redlands
esrihq = geocode("380 New York St", redlands['extent'])[0]
map.draw(esrihq['location'])

In [62]:
redlands['extent']

{'xmax': -117.108538,
 'xmin': -117.256538,
 'ymax': 34.129567,
 'ymin': 33.981567}

<a id="proximity-searches"></a>
## Proximity searches

Geocoding results are typically sorted according to their relevance to the search and their relative importance. However, with some applications, especially mobile apps, users are more concerned with finding features closest to their current location. For this reason, the geocode method supports prioritization of candidates based on their distance from a specified point. By passing in the location and distance parameters, you can define a circular area of influence for your searches. The location parameter represents the center point of the area, and the distance parameter defines the radius in meters. The purpose of this area is to boost the rank of results closest to the specified location so that they show up higher in the list of candidates. Results that are within the area receive a greater boost than those outside the area.


### Example: Find a place name using proximity search (Golden Nugget)

It is important to note that proximity search does not filter results that are outside of the location and distance area—it is intended to influence the sort order of results so that the most locationally relevant candidates are returned first. For instance, if your location is in Las Vegas, distance=3000, and you search for "Golden Nugget", the first candidate is Golden Nugget in Las Vegas. The second is Golden Nugget in Biloxi, Mississippi. So even though Golden Nugget in Biloxi, Mississippi is much farther away than the 3000 meter search distance, it is still returned because it is the second most relevant (closest) candidate. In general, the number of candidates returned by a proximity search is only limited by the maxLocations parameter.

In [63]:
map = gis.map("Las Vegas")
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_12.png)

In [64]:
lasvegas = geocode("Las Vegas, NV")[0]
lasvegas['location']

{'x': -115.1372160299997, 'y': 36.174968712000464}

We search for Golden Nugget near Las Vegas, NV and plot it on the map:

In [65]:
goldennuggets = geocode("Golden Nugget", location=lasvegas['location'], distance=3000)

In [66]:
map.draw(goldennuggets[0]['location'])

In [67]:
# print the matched addresses - note that the address in Las Vegas is the first result
for nugget in goldennuggets:
    print(nugget['attributes']['Place_addr'])

129 Fremont St, Las Vegas, Nevada
151 Beach Blvd, Biloxi, Mississippi
Mudalige Mawatha, Colombo, Western
2406 W Diversey Ave, Chicago, Illinois
Huron Ave, Atlantic City, New Jersey
20 Marine Parade, Great Yarmouth, Norfolk
Marco-Polo-Straße 3, Satteldorf, Baden-Württemberg
Brixner Straße 4, Innsbruck, Tirol
Rainerstraße 19, Linz, Oberösterreich
117 Lonsdale St, Melbourne, Victoria
29 Bannister Rd, Boddington, Western Australia
R348, Ballinasloe, County Galway
22 Shaftesbury Avenue, London
Mallikatta Road, Mangalore, Karnataka
Dr Ramaswamy Salai, Chennai, Tamil Nadu


Contrast the results with a search for Golden Nugget without the location and distance parameters influencing the search results:

In [68]:
goldennuggets = geocode("Golden Nugget")
for nugget in goldennuggets:
    print(nugget['attributes']['Place_addr'])

151 Beach Blvd, Biloxi, Mississippi
Mudalige Mawatha, Colombo, Western
129 Fremont St, Las Vegas, Nevada
2406 W Diversey Ave, Chicago, Illinois
Huron Ave, Atlantic City, New Jersey
20 Marine Parade, Great Yarmouth, Norfolk
Marco-Polo-Straße 3, Satteldorf, Baden-Württemberg
Brixner Straße 4, Innsbruck, Tirol
Rainerstraße 19, Linz, Oberösterreich
29 Bannister Rd, Boddington, Western Australia
117 Lonsdale St, Melbourne, Victoria
22 Shaftesbury Avenue, London
Mallikatta Road, Mangalore, Karnataka
Dr Ramaswamy Salai, Chennai, Tamil Nadu
R348, Ballinasloe, County Galway


### Example: Find a place name using both proximity and search_extent (Golden Nugget)

If you only want to return candidates within a specific area, and sort the candidates according to their proximity to a location, then you need to define a search extent by passing the search_extent parameter in the request along with the location and distance parameters. Consider the Golden Nugget example again. If your location is in Las Vegas and you want to confine your search results to places named Golden Nugget that are within the map extent, then you would need to construct a request with the following parameters:

In [69]:
address = "Golden Nugget"
location = lasvegas['location']
distance =  4000
search_extent = lasvegas['extent']

In [70]:
goldennuggets = geocode(address, search_extent, location, distance)
for nugget in goldennuggets:
    print(nugget['attributes']['Place_addr'])

129 Fremont St, Las Vegas, Nevada


<a id="category-filtering"></a>
## Category filtering

The geocode method supports filtering searches by category values, which represent address and place types. By including the category parameter in a geocode() call you can avoid false positive matches to unexpected place and address types due to ambiguous searches.

Note:
The category parameter is only functional when used with single line addresses. It does not work with the multifield parameters; specifically the address, neighborhood, city, region, subregion, countryCode, and postal parameters.

For example, a user may search for "Mammoth", expecting the service to match to Mammoth Mountain ski resort. However there are many places in the world named "Mammoth", and so the search returns several cities named "Mammoth".

### Example: Search for "Mammoth" without category

In [71]:
mammoths = geocode("Mammoth")
for mammoth in mammoths:
    print(mammoth['address'])

Mammoth, Arizona, United States
Mammoth, California, United States
Mammoth, Utah, United States
Mammoth, Montana, United States
Mammoth, Missouri, United States
Mammoth, West Virginia, United States
Mammoth, Pennsylvania, United States
Mammoth, Wyoming, United States
Mammoth, New Zealand
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth
Mammoth


The solution for this case is to pass the category parameter in the method call. By including category=Ski Resort, all places that are not ski resorts are bypassed by the search, and only ski resorts whose names begin with "Mammoth" are returned.

### Example: Search for "Mammoth" with category=Ski Resort

In [72]:
mammoths = geocode("Mammoth", category="Ski Resort")
for mammoth in mammoths:
    print(mammoth['address'])

Mammoth Mountain
Takamagahara Mammoth
Mammoth Ski Resort


See the topic <a href="/rest/geocode/api-reference/geocoding-category-filtering.htm">Category filtering</a> for complete details.